In [54]:
import os, sys
cwd = os.getcwd()
print(cwd)

/Users/i522408/Desktop/jupyter-Code


In [55]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Lambda
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint

In [56]:

# File paths
TRAIN_TXT = 'TrainingData-set.txt'
TEST_TXT = 'TestingData-set .txt'
EMBEDDING_FILE = 'GoogleNews-vectors-negative300.bin.gz'
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [57]:
X1train = []
X2train = []
Ytrain  = []
test_txt = open("TestingData-set.txt", "r+")
train_txt = open("TrainingData-set.txt", "r+")

print(type(train_txt))


## import the data and get the two x columns an one output one 
for line in train_txt:
    if "1" in line:
        Ytrain.append("1")
    elif "0" in line:
        Ytrain.append("0")
    line = line.split(".")
    #print("line ",line)
    X1train.append(line[0])
    str = line[1]
    str = str[1:]
    str = str.replace('1', '')
    str = str.replace('0', '')
    X2train.append(str)
while("" in X2train ) : 
    X2train.remove("")


  
  

    


<class '_io.TextIOWrapper'>


In [58]:
import nltk
nltk.download('stopwords')
  


## filter out useless words 
stops = set(stopwords.words('english'))





def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    #text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/i522408/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [59]:
#dictionary which has X1train, X2train:
dic = dict(zip(X1train, X2train))
for (x,y) in dic.items():
    print("  X   :  "     +   x   +   "   Y  :  "  + y)
    
    


  X   :  An Asian woman in a blue top and green headscarf smiling widely as another woman rows a boat in the background   Y  :  An Asian woman is running from a dog
  X   :  A boat worker securing line   Y  :  The boat worker is swimming in the ocean
  X   :  A small girl with a necklace is swimming   Y  :  A small girl with a necklace is in the water	

  X   :  a skateboarder skates in the pool   Y  :  A skater is in the pool
  X   :  A couple, who appear to be Indian or Pakistani, walk on a path beside a body of water, the mother carrying a child in a diaper, the father wrapped in a blanket with the logo of the humanitarian organization "Save the Children   Y  :  	A couple carrying a child are walking along water
  X   :  There is a woman holding a baby, along with a man with a save the children bag   Y  :  A man is watching an airplane
  X   :  A family with a baby, the father is wearing a save the children sign   Y  :  A man is wearing something with writing on it
  X   :  A foreig

In [65]:
# Prepare embedding
vocabulary = dict()
train2Dig1 = []
# '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
inverse_vocabulary = ['<unk>']
#vectors can also be instantiated from an existing file on disk in the original Google’s 
#word2vec C format as a KeyedVectors instance
#word2vec = KeyedVectors.load_word2vec_format(datapath("GoogleNews-vectors-negative300.bin"), binary=True)
# dictionary to save the first column of X1train
x1 = dict()
# Iterate over the questions only of both training and test datasets
#docToVec(list):

for index,x in enumerate(X1train):  
        q2n = []  # q2n -> question numbers representation
        for word in text_to_word_list(x):
     # Iterate through the text of both questions of the row
               # Check for unwanted words
                if word in stops and word not in word2vec.vocab:
                    continue        

                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    q2n.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    q2n.append(vocabulary[word])        

        # transfer x1train from list of string to list of numbers
        train2Dig1.append(q2n)



In [67]:
# Prepare embedding
train2Dig2 = []
vocabulary = dict()
# '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
inverse_vocabulary = ['<unk>']
#vectors can also be instantiated from an existing file on disk in the original Google’s 
#word2vec C format as a KeyedVectors instance
#KeyedVectors.load_word2vec_format(datapath("euclidean_vectors.bin"), binary=True)

# Iterate over the questions only of both training and test datasets
#docToVec(list):

for index,x in enumerate(X2train):  
        s2n = []  # s2n -> question numbers representation
        for word in text_to_word_list(x):
     # Iterate through the text of both questions of the row
               # Check for unwanted words
                if word in stops and word not in word2vec.vocab:
                    continue        

                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    s2n.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    s2n.append(vocabulary[word])        

        # transfer x2train from list of string to list of numbers
        train2Dig2.append(s2n)



[[1, 2, 3, 4, 5, 6, 7, 8, 9, 4, 10], [1, 2, 3, 4, 11, 12, 13], [14, 4, 15, 16, 17], [18, 17, 14, 4, 19, 20, 18, 21], [22, 23, 4, 24, 25, 18, 26], [27, 23, 28, 29, 4, 20, 18, 30], [31, 32, 20, 18, 33], [34, 4, 20, 18, 33], [35, 4, 36, 20, 37, 38], [39, 40, 41, 42, 43, 44, 30], [45, 46, 42, 47, 48, 1, 49], [35, 4, 50, 1, 51], [52, 53, 42, 54], [35, 4, 55, 16, 28, 56, 57, 58], [59, 42, 60, 61], [62, 63, 64, 65, 66, 67, 44, 68, 30], [18, 63, 4, 69, 18, 30], [18, 46, 42, 70, 71, 57, 18, 72], [63, 73, 32, 20, 18, 30], [45, 74, 75, 57, 66, 67, 76, 77, 78], [63, 73, 64, 65, 18, 30], [63, 64, 44, 66, 79], [63, 80, 81, 82, 6, 83, 84, 85], [86, 4, 63, 87, 88, 89], [46, 42, 90, 6, 91, 6, 92], [86, 42, 46, 76, 91], [46, 93, 94, 91], [52, 46, 42, 90, 6, 95, 6, 96, 97], [98, 4, 93], [35, 3, 42, 93], [18, 4, 99, 100], [18, 35, 3, 42, 93, 65, 19, 33], [86, 42, 101, 102], [35, 3, 42, 103], [104, 18, 105, 106, 107, 18, 108, 109], [86, 42, 110, 111, 112, 113, 105, 114, 42, 20, 115, 116, 48, 117, 88], [35,

In [68]:
# print two lists of numbers now x_train1,x_train2
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
embeddings[0] = 0  # So that the padding will be ignored

# Build the embedding matrix
for word, index in vocabulary.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)

del word2vec




In [79]:
#print(X1train)
max_seq_length = max(X1train)
print(max_seq_length)
                     
                     

woman and child on trolley car labeled Powell and market and bay and taylor	The woman is on a trolley
